In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import datetime
from imblearn.over_sampling import SMOTE

In [2]:
from zipfile import ZipFile

zip_file = ZipFile('cc_fraud.zip')
df = pd.read_csv(zip_file.open('fraudTrain.csv'))
df.head(1)

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,0,2019-01-01 00:00:18,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,F,561 Perry Cove,...,36.0788,-81.1781,3495,"Psychologist, counselling",1988-03-09,0b242abb623afc578575680df30655b9,1325376018,36.011293,-82.048315,0


In [3]:
df.shape

(1296675, 23)

In [4]:
# df.info()

In [5]:
df = df.set_index(pd.to_datetime(df['trans_date_trans_time'],format= '%Y-%m-%d %H:%M:%S')).sort_index()

In [6]:
df.index.min(),df.index.max()

(Timestamp('2019-01-01 00:00:18'), Timestamp('2020-06-21 12:13:37'))

In [7]:
df.head(1)

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
trans_date_trans_time,,,,,,,,,,,,,,,,,,,,,
2019-01-01 00:00:18,0,2019-01-01 00:00:18,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,F,561 Perry Cove,...,36.0788,-81.1781,3495,"Psychologist, counselling",1988-03-09,0b242abb623afc578575680df30655b9,1325376018,36.011293,-82.048315,0


In [8]:
df['age_in_days'] = (pd.Timestamp.now() - pd.DatetimeIndex(df['dob']))// pd.Timedelta('1d')
# df.head()

In [9]:
# (pd.to_datetime('now') - pd.date_range('19791215',freq='D',periods=365))//pd.Timedelta('1D')

In [10]:
df['dayofweek'] = df.index.day_name()

In [11]:
df['hourofday'] = df.index.hour

In [12]:
df = df.drop(columns=['Unnamed: 0','trans_date_trans_time','cc_num','first','last','street','city','state','trans_num','lat','long','dob','trans_num','unix_time'])

In [13]:
raw = pd.read_csv(zip_file.open('fraudTrain.csv'))
deduplicated = raw.groupby(level=0).first()  # remove duplicates
deduplicated.flags.allows_duplicate_labels = False  # disallow going forward

In [15]:
deduplicated = deduplicated.set_index(pd.to_datetime(deduplicated['trans_date_trans_time'],format= '%Y-%m-%d %H:%M:%S')).sort_index()
# deduplicated.head()

DuplicateLabelError: Index has duplicates.
                              positions
label                                  
2019-01-01 00:32:15            [43, 44]
2019-01-01 01:09:57            [92, 93]
2019-01-01 01:14:11            [96, 97]
2019-01-01 01:34:25          [128, 129]
2019-01-01 01:37:06          [133, 134]
...                                 ...
2020-06-21 08:18:56  [1296242, 1296243]
2020-06-21 10:16:31  [1296454, 1296455]
2020-06-21 10:32:03  [1296476, 1296477]
2020-06-21 10:58:26  [1296526, 1296527]
2020-06-21 11:48:51  [1296622, 1296623]

[21573 rows x 1 columns]

In [ ]:
# raw[raw.duplicated()]
by_min = raw.asfreq('M')

In [ ]:
df[df.index.duplicated()]

In [ ]:
df.groupby(level=0).head()

In [ ]:
df.hourofday.value_counts()

In [ ]:
# plt.plot(df.is_fraud)
from matplotlib.pyplot import figure

figure(figsize=(20, 8), dpi=80)
plt.plot(df.is_fraud.resample('D').mean())

In [ ]:
# notebookbreak

In [ ]:
df.index

In [ ]:
pd.date_range("00:00", "23:59", freq="8H").strftime('%H:%M:%S') #+ pd.Timedelta(5,unit="H")

In [ ]:
test_df = pd.DataFrame(pd.date_range('2000-01-02', freq='1D', periods=15), columns=['Date'])

bins_dt = pd.date_range('2000-01-01', freq='3D', periods=6)
bins_str = bins_dt.astype(str).values

labels = ['({}, {}]'.format(bins_str[i-1], bins_str[i]) for i in range(1, len(bins_str))]

test_df['cat'] = pd.cut(test_df['Date'], bins=bins_dt, labels=labels)

In [ ]:
test_df.head()

In [ ]:
first_shift = pd.date_range(pd.to_datetime('21:00:01'),pd.to_datetime('05:00:00'),periods= 1)
first_shift

In [ ]:
first_shift[0]

In [ ]:
first_shift = pd.date_range('00:00:01','08:00:00', periods= 1)
second_shift = pd.date_range('08:00:01','16:00:00', periods= 1)
overnight_shift = pd.date_range('16:00:01','00:00:00', periods= 1)
bin_times = [first_shift[0],second_shift[0],overnight_shift[0]]
bin_labels = ['first_shift','second_shift']

In [ ]:
df['time_bin'] = pd.cut(df.index, bins = bin_times, labels=bin_labels, right=False)

In [ ]:
df.time_bin

In [ ]:
categoricals = ['merchant','category','first','last','gender','street','city','state','job']

In [ ]:
df.head()

In [ ]:
def get_target_and_features(df):
    target = df.columns.to_list()[-1:]
    features = df.columns.to_list()[:-1]
    return target,features

In [ ]:
target,features = get_target_and_features(df)
target

In [ ]:
def train_val(df):
    y = df[target]
    x = df[features]
    x_train, x_validate, y_train, y_validate = train_test_split(x,y,test_size=.30, random_state=42)
    return x_train,x_validate,y_train,y_validate

In [ ]:
x_train,x_validate,y_train,y_validate = train_val(df)

In [ ]:
x_train.sample()

# Need to encode here. (pd.get_dummies)

In [ ]:
features

In [ ]:
for_dummies = ['merchant','category','gender','zip','job','age']

In [ ]:
dummy_train = pd.get_dummies([])
dummy_val
# dummy_test - need to create test df since this was previously split and exists as its own dataset

In [ ]:
xtrain_smote = df[['lat','long','amt','city_pop','merch_lat','merch_long']]
xvalidate_smote = df[['lat','long','amt','city_pop','merch_lat','merch_long']]
xtrain_smote.sample()

In [ ]:
y = df[target]
x = df[features]
x_train, x_validate, y_train, y_validate = train_val(df)

In [ ]:
x_train.head(1)

In [ ]:
sns.histplot(y_train)
plt.title("Class Imbalance: is_Fraud")

In [ ]:
y_train.value_counts()

In [ ]:
from imblearn.over_sampling import SMOTENC

sm = SMOTENC(random_state = 42,categorical_features = categoricals)
xtrain_resample, ytrain_resample = sm.fit_resample(x_train, y_train)
xval_resample, yval_resample = sm.fit_resample(x_validate,y_validate)

In [ ]:
from imblearn.over_sampling import SMOTE

sm = SMOTE(random_state = 42)
xtrain_resample, ytrain_resample = sm.fit_resample(x_train, y_train)
xval_resample, yval_resample = sm.fit_resample(x_validate,y_validate)

In [ ]:
plot_resample(x_train,y_train,xtrain_resample,ytrain_resample,"SMOTE REBALANCING")